Shivangi Kochrekar
<br> A 2018130020

##Experiment 3

**Objective:**

(i) Apply Naive bayes classifier for given data see Naive_bayes_handout.zip file in edmodo.

(ii) Apply and compare Linear Discriminating Analysis with Naive bayes classifiers w.r.t. error, specificity and sensitivity

Load the construct an python code for:

1. Convert the textual meta-data into a suitable (e.g. corpus) object.

2. Triage some of the irrelevant punctuation and other symbols in the corpus document,change all text to lower case, etc.

3. Tokenize the job descriptions into words. Examine the distributions of two important features

4. Classify attributes in two categories.

5. Graphically visualize the difference between low and high value of targeted feature graph.

6. Transform the features into categorical data

7. Ignore those low frequency words and report the sparsity of your categorical data matrix with or without delete those low frequency words. Note that the sparsity of a matrix is the fraction:

Sparsity(A) = number of zero-valued elements / total number of matrix elements (m×n).

1. Apply the Naive Bayes classifier to original matrix and lower dimension matrix, what do you observe?

2. Apply and compare LDA and Naive Bayes classifiers with respect to the error, specificity and sensitivity.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Datasets/IMDB/IMDB_Dataset.csv.zip"

Archive:  /content/drive/MyDrive/Datasets/IMDB/IMDB_Dataset.csv.zip
  inflating: IMDB Dataset.csv        


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from nltk.tokenize.toktok import ToktokTokenizer

In [ ]:
#Reading the dataset 
imdb_data = pd.read_csv("/content/IMDB Dataset.csv")
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
imdb_data = imdb_data.iloc[:1000,:]

In [ ]:
imdb_data.shape

(1000, 2)

In [ ]:
#sentiment count
imdb_data['sentiment'].value_counts()

positive    501
negative    499
Name: sentiment, dtype: int64

In [ ]:
#split the dataset  
#train dataset
train_reviews=imdb_data.review[:800]
train_sentiments=imdb_data.sentiment[:800]
#test dataset
test_reviews=imdb_data.review[800:]
test_sentiments=imdb_data.sentiment[800:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(800,) (800,)
(200,) (200,)


In [ ]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

In [ ]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

In [ ]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

In [ ]:
imdb_data.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,A wonder littl product the film techniqu is ve...,positive
2,I thought thi wa a wonder way to spend time on...,positive
3,basic there a famili where a littl boy jake th...,negative
4,petter mattei love in the time of money is a v...,positive


In [ ]:
imdb_data['review'] = imdb_data['review'].str.lower()
imdb_data.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,a wonder littl product the film techniqu is ve...,positive
2,i thought thi wa a wonder way to spend time on...,positive
3,basic there a famili where a littl boy jake th...,negative
4,petter mattei love in the time of money is a v...,positive


In [ ]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=True):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

{'further', 'out', "you're", 'll', 'were', 'an', 'just', 'being', 'is', 'through', 've', "doesn't", 'that', 'hadn', 'himself', "you'll", "she's", 'down', 's', 'these', 'he', 'during', 'off', 'its', 'can', 'most', "don't", 'not', 'shouldn', 'with', 'than', 'in', "you'd", 'has', "mightn't", 'had', "didn't", 'this', 'after', 'other', 'ain', 'both', 'some', 'aren', 'up', 'why', 'yourselves', 'the', 'hasn', 'for', 'me', 'more', 'now', 'she', 'ourselves', 'as', 'him', 'mustn', 'needn', "hasn't", 'while', 'weren', 'you', "shouldn't", 'because', 'should', 'about', 'or', 'couldn', 'no', 'doesn', 'here', "won't", 'to', 'who', 'from', "weren't", 'them', 'between', "haven't", "isn't", 'haven', 'until', 'themselves', 'o', 'does', 'against', 'own', 'there', 'which', 'won', 'it', 'his', 'yours', 'do', "it's", 'where', 'doing', 'a', 'any', 'm', 'over', 'yourself', 'again', 'all', "needn't", 'my', 'hers', 'their', 'ma', "couldn't", 'we', 'on', 'only', 'are', 'her', 'under', 'have', 'by', 'myself', 'am'

In [ ]:
#normalized train reviews
norm_train_reviews=imdb_data.review[:800]
norm_train_reviews[0]

'one review ha mention watch 1 oz episod youll hook right thi exactli happen meth first thing struck oz wa brutal unflinch scene violenc set right word go trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordit call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti wa surreal couldnt say wa readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison exp

In [ ]:
#Normalized test reviews
norm_test_reviews=imdb_data.review[800:]
norm_test_reviews[890]

'salva hi pal bigardo margin law dure live see panhandl car underground pitch get donat lame one give anyth salva harden petti crimin doesnt even ani redeem qualiti discov realiti show televis give idea next religion show busi prove win combin someth salva capit onh bigardo jail accident death priest wa critic duo salva show natur realiti show transform christlik figur instant success program esp nononsens woman show produc cant escap way salva pay unusu attent ultim salva victim hi success endjordi molla whose first directori job thi movi success way film satir role televis spain wa vulner type program ha seen share bizarr director felt assault view public want set hi stori point view peopl make fortun naiv audienceth ensembl cast ha good moment film mr molla like ani actor decid direct hi first featur would effect concentr pictur front camera candela pena good actress one best reason watch movi juan carlo villedo david gimenez cacho franco francescoantonio florinda chico rest respond

Bags of words model

It is used to convert text documents to numerical vectors or bag of words.

In [ ]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train_reviews)
#transformed test reviews
cv_test_reviews=cv.transform(norm_test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

BOW_cv_train: (800, 186459)
BOW_cv_test: (200, 186459)


Term Frequency-Inverse Document Frequency model (TFIDF)

It is used to convert text documents to matrix of tfidf features.

In [ ]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (800, 186459)
Tfidf_test: (200, 186459)


In [ ]:
#labeling the sentiment data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

(1000, 1)


In [ ]:
#Spliting the sentiment data
train_sentiments=sentiment_data[:800]
test_sentiments=sentiment_data[800:]
print(train_sentiments)
print(test_sentiments)

[[1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]


In [ ]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

MultinomialNB()
MultinomialNB()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

[1 1 0 1 0 0 0 0 1 0 0 1 1 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 0 0
 0 0 0 1 0 1 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 1
 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 1 1 0 1 0 1 0 1 0 1 0
 0 0 1 1 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 1 0 0 0 1 0 0 1 0
 1 1 1 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 1 1 0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 1
 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0]
[1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0
 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 1 0 0 0 1 0 0 0 1 0
 0 0 1 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 1 0 1 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0
 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0]


In [ ]:
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

mnb_bow_score : 0.6
mnb_tfidf_score : 0.54


In [ ]:
#Classification report for bag of words 
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)
#Classification report for tfidf features
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.54      0.69      0.60        89
    Negative       0.68      0.53      0.60       111

    accuracy                           0.60       200
   macro avg       0.61      0.61      0.60       200
weighted avg       0.62      0.60      0.60       200

              precision    recall  f1-score   support

    Positive       0.49      0.82      0.61        89
    Negative       0.69      0.32      0.43       111

    accuracy                           0.54       200
   macro avg       0.59      0.57      0.52       200
weighted avg       0.60      0.54      0.51       200



In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=1)

In [ ]:
lda.fit(cv_train_reviews.todense(), train_sentiments)
print(f"Training Score: {lda.score(cv_train_reviews,train_sentiments)}")
print(f"Test Score: {lda.score(cv_test_reviews,test_sentiments)}")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Score: 0.885
Test Score: 0.58


#Conclusion:
Preprocessing operations such as cleaning, tokenization, lemmatization and conversion to document term matrix using vectorizer for training classification model was performed on the data.
The Naive Bayes model outperformed LDA on this dataset 